In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import h5py
import numpy as np
from keras.models import load_model
import matplotlib.pyplot as plt

import pylab
pylab.rcParams['figure.figsize'] = (10, 10)

from unveiler import Model
from metrics import fmeasure, recall, precision

N = 20
direction = ['NW', 'W', 'SW', 'N', 'Stand', 'S', 'NE', 'E', 'SE', 'Undefined']
if __name__ == "__main__":
    
    model_file = '../../reinforcement/models/rgb/robot_model.h5'
#     model_file = 'models/rgb/robot_model.h5'
    data_file = '../mixed_data/2018-02-25-20-51-03_rgb_3300.h5'
    data_file = 'xperia.h5'
    
    keras_model = load_model(model_file, 
         {'fmeasure': fmeasure, 'recall': recall, 'precision': precision})
    keras_model.summary()
    
    with h5py.File(data_file, 'r') as f:
        states = f['states'][:]

#     states = np.expand_dims(np.load('file.npy'), axis=0)
#     states = np.expand_dims(states, axis=0)
#     print(states.shape)
        
    model = Model(keras_model)

    indices = np.random.permutation(states.shape[0])[:N]
    print(indices)
    

In [ ]:
for index in indices:
    state = states[index]

    fig, ax = plt.subplots(1, 3)
    image = np.uint8(state.reshape((9, 128, 128)) * 128 + 128)
    image = image.transpose((1,2,0))

    for i_ax in range(ax.shape[0]):
        ax[i_ax].axis('off')

    ax[0].set_title('i[t-2]')
    ax[0].imshow(image[:,:,0:3])
    ax[1].set_title('i[t-1]')
    ax[1].imshow(image[:,:,3:6])
    ax[2].set_title('i[t]')
    ax[2].imshow(image[:,:,6:9])
    plt.show()

    print('Feeforwarding through the network')
    print(direction[np.argmax(model.predict(state))])
    print(direction[np.argmax(keras_model.predict(np.expand_dims(state, axis=0)))])

    print('Visualizing all activations')
    model.visualize(until=6, n_cols=4) # Stop on last BatchNorm

#         print('Deconvolving first layer')
#         model.deconvolve(index=0)

#         print('Deconvolving second layer')
#         model.deconvolve(index=1)

#         print('Deconvolving third layer')
#         model.deconvolve(index=2)

#         print('Deconvolving fourth layer')
#         model.deconvolve(index=3)